In [ ]:
import tensorflow 
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D,MaxPooling2D, LeakyReLU, PReLU 

In [ ]:
#Class PNet():
    #P-Net


    # Data Preprocessing
    # before passing your data into a neural network you normalize the data by Scaling it

    # scaling the data 
    #X = X/255.0

    #you can also use tf.keras.utils.normalize(X) 
    x = layers.Dense(64, activation='relu')(inputs)
    x = layers.Dense(64, activation='relu')(x)
    predictions = layers.Dense(10, activation='softmax')(x)
    # Convoluted neural network

    def p_net():
        inputs = tf.keras.Input(shape=(12,12,3))
        
        x = layers.Conv2D(10,(3,3),strides=1,padding='valid',name='conv1')(input)
        x = layers.PReLU(shared_axes=[1,2],name='prelu1')(x)
        x = layers.MaxPooling2D(pool_size=2, strides=2)(x)
        
        x = layers.Conv2D(16,(3,3),strides=1,padding='valid',name='conv2')(x)
        x = layers.PReLU(shared_axes=[1,2],name='prelu2')(x)
        
        x = layers.Conv2D(32,(3,3),strides=1,padding='valid',name='conv3')(x)
        x = layers.PReLU(shared_axes=[1,2],name='prelu3')(x)
        
        classifier = layers.Conv2D(2, (1, 1), activation='softmax',name='classifier1')(x)
        classifier = Reshape((2,))(classifier)   # this layer has to be deleted in order to enalbe arbitraty shape input
        
        bbox_regress = Conv2D(4, (1, 1),name='bbox1')(x)
        bbox_regress = Reshape((4,))(bbox_regress)
        
        landmark = Conv2D(10, (1, 1),name='bbox1')(x)
        landmark = Reshape((10,))(landmark)
        
        model = tf.keras.Model(inputs=inputs, outputs=[classifier,bbox_regress,landmark])
        model.compile(optimizer=tf.train.AdamOptimizer(0.00001), loss='categorical_crossentropy', metrics=['accuracy'])
        
        my_adam = adam(lr = 0.00001)

        
                         
                         
    #R-Net

    def r_net():
        model = Sequential() #there are 2 types of models but this is the most common
        model.add(Conv2D(28, (3, 3), strides=1, name='conv1', input_shape =(24, 24, 3)))
        model.add(PReLU(name='prelu1'))
        model.add(MaxPooling2D(pool_size=(3, 3), strides=(2,2)))

        model.add(Conv2D(48, (3, 3), strides=1, name='conv2'))
        model.add(PReLU(name='prelu2'))
        model.add(MaxPooling2D(pool_size=(3, 3), strides=(2,2)))

        model.add(Conv2D(64, (2, 2), strides=1, name='conv3'))
        model.add(PReLU(name='prelu3'))

        model.add(Dense(128))
        model.add(PReLU(name='prelu4'))

        model.add(Dense(2))
        model.add(Activation('softmax'))

        model.add(Dense(4))#THere is something being used input_layer_name='prelu4)

    #O-Net

    def o_net():
        model = Sequential() #there are 2 types of models but this is the most common
        model.add(Conv2D(32, (3, 3), strides=1, name='conv1'), input_shape =(48, 48, 3))
        model.add(PReLU(name='prelu1'))
        model.add(MaxPooling2D(pool_size=(3, 3), strides=(2,2)))

        model.add(Conv2D(64, (3, 3), strides=1, name='conv2'))
        model.add(PReLU(name='prelu2'))
        model.add(MaxPooling2D(pool_size=(3, 3), strides=(2,2)))

        model.add(Conv2D(64, (3, 3), strides=1, name='conv3'))
        model.add(PReLU(name='prelu3'))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

        model.add(Conv2D(128, (2, 2), strides=1, name='conv4'))
        model.add(PReLU(name='prelu4'))

        model.add(Dense(256))
        model.add(PReLU(name='prelu5'))

        model.add(Dense(2))
        model.add(Activation('softmax'))

        model.add(Dense(4))#THere is something being used input_layer_name='prelu5

        model.add(Dense(10))#THere is something being used input_layer_name='prelu5